In [1]:
# libraries 
import pandas as pd 
import os
import numpy as np
import time
import random
from csv import reader
import time

# plots:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# scraping
from bs4 import BeautifulSoup
from lxml import etree #for using XPATH with beautifulsoup
import requests
import numpy as np

#JSON
import json

# GeoJSON
# import geopandas as gpd

# regular expression
import re 

# concurrent futures - boosts the process of scraping utilicing the CPU better
# two main classes- 
#       1) executor class: manages all the threads and workload
#       2) futures class: creates a little instance and manages data coming back
import concurrent.futures

# libraries: 
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# The plan :


* filter for properties that mach the following 
    * **housing type:** Villa (detached house) , Rækkehus (raðhús) , Ejerlejlighed (condo)
    * **Time:** 2012-2023
    * **Sale type:** normal or according to boliga.dk Almindeligt frit salg
    * ***according to Boliga.dk the result is 757.950 properties*** 
    
* First I scrape the API that gives me search result 
    * they dont seem to filter i.e. the properties amounts to 757.950 and have 50 properties per page which means **15.801** pages which they allow to search in. 
    
    * parameters that it takes are : 
        * searchTab: 1
        * propertyType: 1,2,3
        * saleType: 1
        * salesDateMin: 2012
        * salesDateMax: 2023
        * sort: date-d
        * page: 2
        
    * Property types are numbers such that 
        * property type = 1 is **Villa**
        * property type = 2 is **Rækkehus**
        * property type = 3 is **Ejerlejlighed**
        * property type = 6 is **Landejendom** (farm house) 


# Setting up User agent and header 

In [2]:
# change user agent for each request randomly
def get_user_agent():
    user_agent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.57'
    ]
    user_agent = user_agent_list[random.randint(0,len(user_agent_list)-1)]
   
    return user_agent

def get_header():
    
    # get user-agent
    user_agent = get_user_agent()

     # set up header     
    header = {
        'authority': 'api.boligsiden.dk',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'origin': 'https://www.boligsiden.dk',
        'referer': 'https://www.boliga.dk/',
        'sec-ch-ua': '"Microsoft Edge";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': user_agent
#         'x-api-key': 'GWD0fljZzkc8GOLV',
    }
    return header 

# scrape page 

In [ ]:
# def scrape_page(page):
#     global dataframes_list
    
#     print('-------------------------------------')
#     print(f'---- page number {page} of (510.670 boliger solgt')
#     print('-------------------------------------')

#     # Set up the header
#     ########################
#     main_header = get_header()

#     # Set up the url of the API
#     ###########################
#     main_url = f'https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&propertyType=1,2,3,6&saleType=1&salesDateMin=2012&salesDateMax=2023&sort=date-d&page={page}'

#     # Make the HTTPS Request 
#     ########################
#     response = session.get(main_url, headers=main_header)

#     if response.status_code == 200:
        
#         print('--------------------- scraping !')

#         # Convert to JSON 
#         #################
#         page_result = response.json()['results']

#         # loop over each object and construct the dataframe
#         ####################################################
#         dataframes = [pd.DataFrame([prop]) for prop in page_result]

#         # append to the dataframes_list 
#         ###############################
#         dataframes_list.extend(dataframes)
#     else:
#         print()
#         print('############################################################')
#         print('------------------- BAD REQUEST, NOT 200 ------------------')
#         print('############################################################')
#         print()

    
    

In [5]:
################################## for years 2006 to 2011

def scrape_page(page):
    global dataframes_list
    global session
    
    print('-------------------------------------')
    print(f'---- page number {page} of 10.214 pages (510.670 boliger solgt')
    print('-------------------------------------')

    # Set up the header
    ########################
    main_header = get_header()

    # Set up the url of the API
    ###########################
    main_url = f'https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&propertyType=1,2,3,6&saleType=1&salesDateMin=2006&salesDateMax=2011&sort=date-d&page={page}'

    # Make the HTTPS Request 
    ########################
    response = session.get(main_url, headers=main_header)

    if response.status_code == 200:
        
        print('--------------------- scraping !')

        # Convert to JSON 
        #################
        page_result = response.json()['results']

        # loop over each object and construct the dataframe
        ####################################################
        dataframes = [pd.DataFrame([prop]) for prop in page_result]

        # append to the dataframes_list 
        ###############################
        dataframes_list.extend(dataframes)
    else:
        print()
        print('############################################################')
        print('------------------- BAD REQUEST, NOT 200 ------------------')
        print('############################################################')
        print()

    
    

10213.4

# Main function 

The total number of pages is 15.159.

In [6]:

###############################################
# for loop: 
# --------- go over each of the  15.801 pages
###############################################

dataframes_list = []

session = requests.Session()

page_numbers = list(range(1,10214+1))

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor: 
    retry = Retry(connect=3, backoff_factor=0.5) # max 3 retries to the same link with a 0.5 sec delay
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    # run script
    executor.map(scrape_page,page_numbers)
    
    
# ###################################################################################################
# # MERGE THE DATAFRAME THEN SAVE THE FILE
# ###################################################################################################
    
# print('##########################################################')
# print('------------------- MERGIN STARTING -----------------------')
# print('##########################################################')
    

# def merge_tables(table):
#     global bigdata
#     global count 
#     count +=1
    
#     print(f"Table {count} out of 757.950")
#     bigdata = pd.concat([bigdata, table], ignore_index=True)
    

# # Initialize an empty DataFrame to store the concatenated data
# bigdata = pd.DataFrame()

# count = 0

# with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor: 
#     # run script
#     executor.map(merge_tables,dataframes_list)


# #############################################################################
# # Afer main forloop: 
# # -----------------
# # --> construct the main dataframe by merging all the dataframes in the list 
# #############################################################################
# # main_df = pd.concat(dataframes_list, ignore_index=True)

# #####################
# # Save result as CSV 
# #####################

# print('##########################################################')
# print('------------------- SAVING DATA TO CSV FILE -----------------------')
# print('##########################################################')

# path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_1_Scrape_Property_ID_Links\Boliga_Step_1_property_links.csv'

# bigdata.to_csv(path, encoding='utf-8',index=False)
    
    
# # ------------------------------------------------ same as above but using for loop !    
    
# for page in range(1,15801+1):
    
#     print('-------------------------------------')
#     print(f'---- page number {page} of 15.159')
#     print('-------------------------------------')
    
#     # Set up the header
#     ########################
#     main_header = get_header()
    
#     # Set up the url of the API
#     ###########################
#     main_url = f'https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&propertyType=1,2,3,6&saleType=1&salesDateMin=2012&salesDateMax=2023&sort=date-d&page={page}'
    
#     # Make the HTTPS Request 
#     ########################
#     response = session.get(url,headers=main_header)
    
#     if response.status_code == 200:

#         # Convert to JSON 
#         #################
#         page_result = response.json()['results']

#         # loop over each object and construct the dataframe
#         ####################################################
#         dataframes = [pd.DataFrame([prop]) for prop in page_result]

#         # append to the dataframes_list 
#         ###############################
#         dataframes_list.extend(dataframes)
#     else:
#         print()
#         print('############################################################')
#         print('------------------- BAD REQUEST, NOT 200 ------------------')
#         print('############################################################')
#         print()
    
        


# #############################################################################
# # Afer main forloop: 
# # -----------------
# # --> construct the main dataframe by merging all the dataframes in the list 
# #############################################################################
# main_df = pd.concat(dataframes_list, ignore_index=True)

# #####################
# # Save result as CSV 
# #####################

# path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_1_Scrape_Property_ID_Links\Boliga_Step_1_property_links.csv'

# main_df.to_csv(path, encoding='utf-8',index=False)

-------------------------------------
---- page number 1 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 2 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 3 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 4 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 5 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 6 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 7 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 8 of 10

--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 54 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 55 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 56 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !-------------------------------------
---- page number 57 of 10.214 pages (510.670 boliger solgt
-------------------------------------

-------------------------------------
---- page number 58 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 59 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scr

-------------------------------------
---- page number 104 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 105 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 106 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 107 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 108 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 109 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 110 of 10.214 pages (510.670 boliger solgt
--------------------

--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 154 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 155 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
--------------------------------------------------------------------------
---- page number 157 of 10.214 pages (510.670 boliger solgt
-------------------------------------

---- page number 156 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 158 o

--------------------- scraping !
-------------------------------------
---- page number 204 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 205 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 206 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 207 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 208 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- s

...

-------------------------------------
---- page number 10016 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10017 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 10018 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10019 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10020 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10021 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------

--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 10065 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10066 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10067 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 10068 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10069 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10070 of 10.214 pages (5

-------------------------------------
---- page number 10115 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10116 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10117 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10118 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10119 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10120 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------

--------------------- scraping !
-------------------------------------
---- page number 10164 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10165 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10166 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
-------------------------------------
---- page number 10167 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 10168 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------------------------------------
---- page number 10169 of 10.214 pages (510.670 boliger solgt
-------------------------------------
-------

--------------------- scraping !
-------------------------------------
---- page number 10213 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
-------------------------------------
---- page number 10214 of 10.214 pages (510.670 boliger solgt
-------------------------------------
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !
--------------------- scraping !


In [ ]:
len(dataframes_list)

# ---- Save data as multiple chunks 

In [7]:
# Assuming you have a list of DataFrames called "db_list"

# Specify the chunk size (adjust based on your system's available memory)
chunk_size = 10000  # Process 1000 DataFrames at a time

# Initialize a counter to keep track of the chunks
counter = 81

for i in range(0, len(dataframes_list), chunk_size):
    print(f" -------- chunk {counter} out of 80")
    
    chunk = dataframes_list[i:i + chunk_size]
    combined_df = pd.concat(chunk, ignore_index=True)
    
    # Save each chunk to a CSV file with a unique name
    csv_filename = f'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_1_Scrape_Property_ID_Links\data\combined_data_chunk_{counter}.csv'
    combined_df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    counter += 1
    
    

 -------- chunk 81 out of 80
 -------- chunk 82 out of 80
 -------- chunk 83 out of 80
 -------- chunk 84 out of 80
 -------- chunk 85 out of 80
 -------- chunk 86 out of 80
 -------- chunk 87 out of 80
 -------- chunk 88 out of 80
 -------- chunk 89 out of 80
 -------- chunk 90 out of 80
 -------- chunk 91 out of 80
 -------- chunk 92 out of 80
 -------- chunk 93 out of 80
 -------- chunk 94 out of 80
 -------- chunk 95 out of 80
 -------- chunk 96 out of 80
 -------- chunk 97 out of 80
 -------- chunk 98 out of 80
 -------- chunk 99 out of 80
 -------- chunk 100 out of 80
 -------- chunk 101 out of 80
 -------- chunk 102 out of 80
 -------- chunk 103 out of 80
 -------- chunk 104 out of 80
 -------- chunk 105 out of 80
 -------- chunk 106 out of 80
 -------- chunk 107 out of 80
 -------- chunk 108 out of 80
 -------- chunk 109 out of 80
 -------- chunk 110 out of 80
 -------- chunk 111 out of 80
 -------- chunk 112 out of 80
 -------- chunk 113 out of 80
 -------- chunk 114 out of 80

In [ ]:
790045/10000

In [8]:
listinn = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

cunk = 5

for i in range(0,len(listinn), cunk):
    cunk_list = listinn[i:i+cunk]
    print(cunk_list)

[1, 2, 3, 4, 5]
[6, 7, 8, 9, 10]
[11, 12, 13, 14, 15]


In [ ]:
import csv

path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Step_1_Scrape_Property_ID_Links\list_dataframes.csv' 
with open(path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(dataframes_list)

In [ ]:
dataframes_list.to

In [ ]:
import sys

import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# ---- Here I will get the rest i.e. years 2006-2011

# -------------------- setup up code (white pages )

In [ ]:

page_number = 1

main_header = get_header()

url = f'https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&propertyType=1,2,3,6&saleType=1&salesDateMin=2012&salesDateMax=2023&sort=date-d&page={page_number}'

response = requests.get(
    url,
    headers=main_header,
)

response

In [ ]:
page_result= response.json()['meta']

In [ ]:
page_result

In [ ]:
len(page_result)

In [ ]:
page_result[0]

In [ ]:
gagna_listinn = [pd.DataFrame([prop]) for prop in page_result]
final_df = pd.concat(gagna_listinn, ignore_index=True)
final_df

In [ ]:
gagna_listinn = []

for prop in page_result:
    print(len(prop))
    
    # Construct a DataFrame from the JSON object
    df = pd.DataFrame([prop])

    # Append the DataFrame to the list of DataFrames
    gagna_listinn.append(df)

# Concatenate the list of DataFrames into a single DataFrame
final_df = pd.concat(gagna_listinn, ignore_index=True)
    

In [ ]:
final_df